In [1]:
import pandas as pd

## Preparing the Data for Cancer Sub-Type Classification

In [2]:
df_breast = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/breast.csv')
df_normal = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/normal.csv')

In [3]:
def create_combined_df(cancer_df, normal_df, random_state):
    cancer_count = len(cancer_df)
    normal_sample = normal_df.sample(n=cancer_count, random_state=random_state, replace=False)

    combined_df = pd.concat([cancer_df, normal_sample], ignore_index=True)
    return combined_df

In [4]:
df_breast_cancer = create_combined_df(df_breast, df_normal, 3)
df_breast_cancer.shape

(518, 54677)

In [5]:
df_breast_cancer['type'].unique()

array(['tumoral_basal', 'tumoral_non_BLC', 'tumoral', 'basal', 'HER',
       'cell_line', 'luminal_A', 'luminal_B', 'normal'], dtype=object)

In [6]:
def preprocessing_sub(df):
    
    # Drop cancer_type Columns
    if "cancer_type" in df.columns:
        df = df.drop(columns="cancer_type")

    # Get X,y
    target = 'type'
    X = df.drop(columns=target)
    y = df[target]
    
    return X,y

# Display proccesed data

Xs_b,ys_b = preprocessing_sub(df_breast_cancer)

display(Xs_b,ys_b)

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,10.896695,6.822755,6.208037,8.240948,3.561534,6.565948,5.810279,5.031073,6.843409,3.915450,...,12.908356,12.359587,14.098307,14.195651,3.894551,3.898071,3.580795,3.432257,3.724186,3.638622
1,9.641718,7.363538,6.256256,8.303424,3.372617,7.079188,5.941050,4.874336,8.996115,3.752041,...,12.391813,11.776600,13.674190,13.740015,4.184290,3.840209,3.527928,3.603502,3.917258,3.821375
2,9.978563,8.278789,7.669063,8.051921,3.599057,6.611970,5.845036,4.912140,10.325110,3.532738,...,11.989426,11.389290,13.668495,13.437867,4.223195,3.869195,3.623235,3.426600,3.892071,3.934255
3,11.447405,6.893432,6.376750,7.982946,3.508099,6.286912,6.229778,4.651420,6.239842,3.689191,...,12.304736,11.830111,13.991247,13.855367,4.016240,3.662467,3.405675,3.390272,3.854924,3.672979
4,10.424252,7.406838,6.789227,8.576380,3.622979,6.701194,6.022236,5.067318,8.196237,3.699863,...,11.842361,11.165080,13.562007,13.487130,4.240036,3.926423,3.653918,3.647404,3.969583,3.752808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,10.560966,7.507896,7.998389,8.725496,3.572918,9.601450,5.484624,6.081885,10.979798,4.271587,...,12.590997,11.782307,13.877738,13.760668,10.129098,8.006231,9.042986,3.515665,4.039623,3.896915
514,10.783435,6.978394,6.767972,5.566966,2.205268,8.291196,5.970484,5.249955,5.441837,5.271417,...,13.250909,13.238642,14.516039,14.365754,2.688534,2.413363,2.357578,2.175542,2.414312,2.465603
515,10.024829,5.895326,5.837347,10.219339,2.669769,7.325551,5.267367,4.829835,5.965249,3.702102,...,12.761072,12.031207,13.974419,13.835602,9.835593,7.515038,8.603459,2.507644,2.849954,2.849750
516,11.328720,6.243838,5.533624,8.317443,3.569423,8.545089,5.951187,5.044808,7.037779,3.550192,...,12.851080,12.366678,14.061923,13.996265,10.062664,5.686620,8.320798,3.353953,3.871147,3.688010


0      tumoral_basal
1      tumoral_basal
2      tumoral_basal
3      tumoral_basal
4      tumoral_basal
           ...      
513           normal
514           normal
515           normal
516           normal
517           normal
Name: type, Length: 518, dtype: object

## Sub-Type Classification using Logistic Regression

In [7]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def kfold_with_pca(X, y, n_components, n_splits=5):
    
    # Ensure consistent column ordering
    feature_names = X.columns.tolist()

    # Get the unique class labels
    unique_classes = np.unique(y)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    pca = PCA(n_components=n_components)
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Lists to store fold metrics
    accuracies, recalls, precisions, f1_scores = [], [], [], []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply PCA
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        # Train the model
        model.fit(X_train_pca, y_train)

        # Predict on the test set
        y_pred = model.predict(X_test_pca)

        # Calculate metrics
        accuracies.append(accuracy_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        precisions.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred, average='macro'))

    # Final PCA fitting and model training on full data
    X_pca_full = pca.fit_transform(X)
    model.fit(X_pca_full, y)

    # Aggregate metrics
    metrics = {
        "Mean Accuracy": np.mean(accuracies),
        "Mean Recall": np.mean(recalls),
        "Mean Precision": np.mean(precisions),
        "Mean F1 Score": np.mean(f1_scores),
    }

    # Print cross-validation results
    print(f"5-Fold CV Results with {n_components} PCA components:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

    # Return the final model, PCA object, and metrics
    return {
        "model": model,
        "pca": pca,
        "feature_names": feature_names,
        "metrics": metrics,
    }

In [11]:
results = kfold_with_pca(Xs_b, ys_b, n_components=50)
results


5-Fold CV Results with 50 PCA components:
Mean Accuracy: 0.95
Mean Recall: 0.92
Mean Precision: 0.94
Mean F1 Score: 0.92


{'model': RandomForestClassifier(random_state=42),
 'pca': PCA(n_components=50),
 'feature_names': ['1007_s_at',
  '1053_at',
  '117_at',
  '121_at',
  '1255_g_at',
  '1294_at',
  '1316_at',
  '1320_at',
  '1405_i_at',
  '1431_at',
  '1438_at',
  '1487_at',
  '1494_f_at',
  '1552256_a_at',
  '1552257_a_at',
  '1552258_at',
  '1552261_at',
  '1552263_at',
  '1552264_a_at',
  '1552266_at',
  '1552269_at',
  '1552271_at',
  '1552272_a_at',
  '1552274_at',
  '1552275_s_at',
  '1552276_a_at',
  '1552277_a_at',
  '1552278_a_at',
  '1552279_a_at',
  '1552280_at',
  '1552281_at',
  '1552283_s_at',
  '1552286_at',
  '1552287_s_at',
  '1552288_at',
  '1552289_a_at',
  '1552291_at',
  '1552293_at',
  '1552295_a_at',
  '1552296_at',
  '1552299_at',
  '1552301_a_at',
  '1552302_at',
  '1552303_a_at',
  '1552304_at',
  '1552306_at',
  '1552307_a_at',
  '1552309_a_at',
  '1552310_at',
  '1552311_a_at',
  '1552312_a_at',
  '1552314_a_at',
  '1552315_at',
  '1552316_a_at',
  '1552318_at',
  '1552319_a_

In [12]:
model = results["model"]
pca = results["pca"]

In [13]:
df_test = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/test_data.csv')
df_breast_test = df_test[df_test['cancer_type'].isin(['normal', 'breast'])]

In [14]:
X_test,y_test = preprocessing_sub(df_breast_test)

In [15]:
from sklearn.metrics import classification_report

def evaluate_model(model, pca, X_test, y_test, unique_classes, feature_names):
    """
    Evaluate the final trained model on test data.
    """
    # Ensure test set columns match the training set
    X_test = X_test[feature_names]

    # Transform test data using PCA
    X_test_pca = pca.transform(X_test)

    # Predict using the final trained model
    y_pred = model.predict(X_test_pca)

    # Calculate overall accuracy
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average=None, labels=unique_classes, zero_division=0)

    # Print results
    print("\nFinal Model Evaluation on Test Dataset:")
    print(f"Accuracy: {accuracy:.2f}")
    print("\nRecall for each class:")
    for i, cls in enumerate(unique_classes):
        print(f"Class {cls}: Recall = {recall[i]:.2f}")

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, labels=unique_classes))


In [16]:
evaluate_model(
    model=results["model"],
    pca=results["pca"],
    X_test=X_test,
    y_test=y_test,
    unique_classes=np.unique(ys_b),
    feature_names=results["feature_names"],
)


Final Model Evaluation on Test Dataset:
Accuracy: 0.94

Recall for each class:
Class HER: Recall = 0.67
Class basal: Recall = 1.00
Class cell_line: Recall = 1.00
Class luminal_A: Recall = 1.00
Class luminal_B: Recall = 1.00
Class normal: Recall = 0.92
Class tumoral: Recall = 1.00
Class tumoral_basal: Recall = 1.00
Class tumoral_non_BLC: Recall = 1.00

Classification Report:
                 precision    recall  f1-score   support

            HER       1.00      0.67      0.80         3
          basal       1.00      1.00      1.00         4
      cell_line       1.00      1.00      1.00         2
      luminal_A       1.00      1.00      1.00         3
      luminal_B       1.00      1.00      1.00         3
         normal       0.98      0.92      0.95        52
        tumoral       0.73      1.00      0.85        11
  tumoral_basal       1.00      1.00      1.00         3
tumoral_non_BLC       1.00      1.00      1.00         1

       accuracy                           0.94    

In [18]:
# Save model

import joblib
joblib.dump(model, 'breast_sub.joblib')
joblib.dump(pca, 'breast_sub_pca.joblib')

['breast_sub_pca.joblib']